In [ ]:
import numpy as np
import time
from data_generator import StudyParameters
from experiment1_solution_quality import DynamicProgramming, StochasticApproximation
import logging
from pprint import pformat

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Create Simple Test Instance

In [ ]:
def create_simple_test_instance():
    """Create test instance with specified parameters."""
    params = StudyParameters(
        T=2,          # 2 booking periods
        N=3,          # 3-day horizon
        C=5,          # 5 rooms capacity
        price_min=50, # Minimum price
        price_max=150,# Maximum price
        alpha=0.1,    # Not used in DP
        beta=0.1      # Not used in DP
    )
    
    # Generate booking classes
    booking_classes = []
    for arrival in range(1, 4):
        for departure in range(arrival, 4):
            booking_classes.append((arrival, departure))
    
    # Uniform arrival probabilities (0.15)
    arrival_probs = {}
    for t in range(1, 3):
        arrival_probs[t] = {bc: 0.15 for bc in booking_classes}
    
    # Uniform price sensitivity (1/max_price)
    epsilon = 1/params.price_max
    price_sensitivity = {bc: epsilon for bc in booking_classes}
    
    return {
        'parameters': params,
        'booking_classes': booking_classes,
        'arrival_probabilities': arrival_probs,
        'reservation_price_params': price_sensitivity
    }

# Compare DP Performance vs. SAA Performance

In [ ]:
def compare_saa_with_dp():
    """
    Compare SAA performance with Dynamic Programming using a simple test instance.
    """
    # Create the same simple test instance
    test_instance = create_simple_test_instance()
    
    logger.info("Test Instance Details:")
    logger.info(f"Time Periods (T): {test_instance['parameters'].T}")
    logger.info(f"Service Horizon (N): {test_instance['parameters'].N}")
    logger.info(f"Room Capacity (C): {test_instance['parameters'].C}")
    logger.info(f"Price Range: [{test_instance['parameters'].price_min}, {test_instance['parameters'].price_max}]")
    logger.info(f"Number of Booking Classes: {len(test_instance['booking_classes'])}")
    
    # Configure SAA parameters for the simple test
    learning_params = {
        'eta_0': 0.5,        # Initial learning rate
        'gamma': 0.05,       # Learning rate decay
        'eta_min': 0.001,    # Minimum learning rate
        'max_epochs': 1000,
        'batch_size': 64     # Batch size
    }
    
    # Run multiple trials of SAA to account for stochasticity
    num_trials = 10
    saa_revenues = []
    saa_times = []
    
    logger.info("\nRunning SAA Trials:")
    for trial in range(num_trials):
        saa = StochasticApproximation(test_instance, learning_params)
        prices, revenue, solve_time = saa.solve()
        
        # Evaluate final solution with more samples
        final_revenue = saa.evaluate(prices, num_samples=10000)
        saa_revenues.append(final_revenue)
        saa_times.append(solve_time)
        
        logger.info(f"Trial {trial + 1}: Revenue = {final_revenue:.4f}, Time = {solve_time:.2f}s")
    
    # Calculate SAA statistics
    avg_revenue = np.mean(saa_revenues)
    std_revenue = np.std(saa_revenues)
    avg_time = np.mean(saa_times)
    
    logger.info("\nSAA Performance Summary:")
    logger.info(f"Average Revenue: {avg_revenue:.4f} (±{std_revenue:.4f})")
    logger.info(f"Average Solve Time: {avg_time:.2f}s")
    
    # Solve using Dynamic Programming for comparison
    dp = DynamicProgramming(test_instance)
    start_time = time.time()
    _, dp_revenue = dp.solve()
    dp_time = time.time() - start_time
    
    logger.info("\nDynamic Programming Results:")
    logger.info(f"Optimal Revenue: {dp_revenue:.4f}")
    logger.info(f"Solve Time: {dp_time:.2f}s")
    
    # Compare performance
    logger.info("\nPerformance Comparison:")
    logger.info(f"Revenue Gap: {((dp_revenue - avg_revenue) / dp_revenue * 100):.2f}%")
    logger.info(f"Time Ratio: {avg_time / dp_time:.2f}x")
    
    return {
        'saa_avg_revenue': avg_revenue,
        'saa_std_revenue': std_revenue,
        'saa_avg_time': avg_time,
        'dp_revenue': dp_revenue,
        'dp_time': dp_time,
        'saa_trials': saa_revenues
    }

In [ ]:
if __name__ == "__main__":
    results = compare_saa_with_dp()

# Test Dynamic Programming Only

In [ ]:
def main():
    test_instance = create_simple_test_instance()
    
    # Print instance details
    logger.info("Test Instance Details:")
    logger.info(f"Time Periods (T): {test_instance['parameters'].T}")
    logger.info(f"Service Horizon (N): {test_instance['parameters'].N}")
    logger.info(f"Room Capacity (C): {test_instance['parameters'].C}")
    logger.info(f"Price Range: [{test_instance['parameters'].price_min}, {test_instance['parameters'].price_max}]")
    
    # Solve using Dynamic Programming
    dp = DynamicProgramming(test_instance)
    results, optimal_value = dp.solve()
    
    # Print formatted results for states with capacity 5
    logger.info("\nValue Function for States with Capacity 5:")
    logger.info(pformat(results))
    
    logger.info(f"\nOptimal Value: {optimal_value:.4f}")

In [ ]:
if __name__ == "__main__":
    main()